.. gps-receivers:

## GPS Receivers

Most GPS receivers have data logging capabilities that you can use with Pipecat to view navigational information.  Some receivers connect to your computer via a serial port or a serial-over-USB cable that acts like a traditional serial port.  Others can push data to a network socket.  For this demonstration, we will receive GPS data sent from an iPhone to a UDP socket:

In [1]:
# nbconvert: hide
from __future__ import absolute_import, division, print_function

import pipecat.test
socket = pipecat.test.mock_socket()

path = "../data/gps"
client = "172.10.0.20"

socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, stop=6)

In [2]:
import pipecat.record
import pipecat.udp
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
for record in pipe:
    pipecat.record.dump(record)

client: 172.10.0.20
string: $GPTXT,01,01,07,Pipecat*12


client: 172.10.0.20
string: $GPGGA,164100,3511.33136,N,10643.48435,W,1,8,0.9,1654.0,M,46.9,M,0,2*50


client: 172.10.0.20
string: $GPRMC,164100,A,3511.33136,N,10643.48435,W,0.00,0.00,311216,003.1,W*7C


client: 172.10.0.20
string: $GPGLL,3511.33136,N,10643.48435,W,164100,A*36


client: 172.10.0.20
string: $HCHDG,129.5,,,8.7,E*29


client: 172.10.0.20
string: $PASHR,164100190,138.24,T,+32.56,+48.49,+00.00,3.141,3.141,35.000,1,0*17




Here, we used :func:`pipcat.udp.receive` to open a UDP socket listening on port 7777 on all available network interfaces ("0.0.0.0") and convert the received messages into Pipecat :ref:`records`, which we dump to the console.  Note that each record includes the IP address of the client (the phone in this case), along with a string containing the raw data of the message.  In this case the raw data is in NMEA format, a widely-used standard for exchanging navigational data.  To decode the contents of each message, we add the appropriate Pipecat device to the end of the pipe:

In [3]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, stop=6)

In [4]:
import pipecat.device.gps
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    pipecat.record.dump(record)

id: GPTXT
text: Pipecat

altitude: 1654.0 meter
dop: 0.9
geoid-height: 46.9 meter
id: GPGGA
latitude: 35.188856 degree
longitude: -106.724739167 degree
quality: 1
satellites: 8
time: 164100

active: True
date: 311216
id: GPRMC
latitude: 35.188856 degree
longitude: -106.724739167 degree
speed: 0.0 knot
time: 164100
track: 0.0 degree
variation: -3.1 degree

active: True
id: GPGLL
latitude: 35.188856 degree
longitude: -106.724739167 degree
time: 164100

heading: 129.5 degree
id: HCHDG
variation: 8.7 degree

heading: 138.24 degree
heading-accuracy: 35.0 degree
heave: 0.0 meter
id: PASHR
pitch: 48.49 degree
pitch-accuracy: 3.141 degree
roll: 32.56 degree
roll-accuracy: 3.141 degree
time: 164100190



As you can see, :func:`pipecat.device.gps.nmea` has converted the raw NMEA messages into records containing human-readable navigational fields with appropriate physical units.  Note that unlike the :ref:`battery-chargers` example, not every record produced by the GPS receiver has the same fields.  The NMEA standard includes many different *types* of messages, and most GPS receivers will produce more than one type.  This will increase the complexity of our code - for example, we will have to test for the presence of a field before extracting it from a record:

In [5]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=110)

In [6]:
import pipecat.device.gps
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    if "latitude" in record:
        print("Latitude:", record["latitude"], "Longitude:", record["longitude"])

Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree


... alternatively, you might key your code off a specific type of message, using the `id` field.

As always, you can convert units safely and explicitly:

In [7]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=120)

In [8]:
import pipecat.device.gps
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    if "speed" in record:
        print(record["speed"].to(pipecat.units.mph))

39.9320468464 mph
40.0586325857 mph
40.1276793526 mph
38.5626193033 mph


In [9]:
# nbconvert: stop

As an aside, you may be wondering at this point why it's necessary to explicitly create the serial port and connect it to `readline` ... why not code that functionality directly into `icharger208b`?  The answer is flexibility: by separating Pipecat's functionality into discrete, well-defined components, those components can be easily combined in new and unexpected ways.  For example, you could use `icharger208b` with a charger that communicated over a network socket instead of a serial port.  Or you could "replay" data from a charger stored in a file: 

Let's explore other things we can do with our pipe.  To begin, you might want to add additional metadata to the records returned from a device.  For example, you might want to append timestamps:

Note that :func:`pipecat.utility.add_timestamp` has added a `timestamp` field to each record.  Timestamps in Pipecat are always recorded using UTC (universal) time, so you will likely want to convert them to your local timezone before formatting them for display:

You could also use :func:`pipecat.utility.add_field` to append your own custom field to every record that passes through the pipe.

Now let's consider calculating some simple statistics, such as the minimum and maximum battery voltage while charging.  When we iterate over the contents of a pipe using a `for` loop, we receive one record at-a-time until the pipe is empty.  We could keep track of a "running" minimum and maximum during iteration, and there are use-cases where that is the best way to solve the problem.  However, for moderately-sized data, Pipecat provides a more convenient approach: 

Here, :func:`pipecat.store.cache` creates an in-memory cache that stores every record it receives.   We have a do-nothing `for` loop that reads data from the charger to populate the cache.  Once that's complete, we can use the cache `table` attribute to retrieve data from the cache using the same keys and syntax we would use with a record.  Unlike a record, the cache returns every value for a given key at once (using a Numpy array), which makes it easy to compute the statistics we're interested in:

Consolidating fields using the cache is also perfect for generating plots with a library like Toyplot (http://toyplot.readthedocs.io):

Note that nothing prevents us from doing useful work in the `for` loop that populates the cache, and nothing prevents us from accessing the cache within the loop.  For example, we might want to display field values from individual records alongside a running average computed from the cache.  Or we might want to update our plot periodically as the loop progresses.

Moving on, you will likely want to store records to disk for later access.  Pipecat provides components to make this easy too.  First, you can add :func:`pipecat.store.pickle.write` to the end of a pipe, to write records to disk using Python's pickle format:

This is another example of the interchangeability of the Pipecat components: the pickle writer is a record consumer, and the pickle reader is a record generator.  In essence, we "broke" our previous pipe into two separate pipes that communicate via the filesystem.  While we won't go into detail here, a similar approach could be used to communicate between threads using a message queue or between processes over a socket.

There is one final issue that we've ignored so far: when to stop logging data.  The :func:`pipecat.utility.readline()` function will read data from the serial port as long as the port is open, blocking indefinitely if no data is arriving.  That means that for all the preceeding examples the `for` loop will never end unless the serial port is closed (i.e. the external device is turned-off or unplugged), or the code is interrupted using Control-C.  While that's fine for prototyping at the command line, we need to have a way to stop collecting data and shutdown cleanly if we're going to automate data logging processes.  Fortunately, Pipecat provides several easy-to-use functions to do just that:

Here, :func:`pipecat.limit.count` ends the loop when `count` records have been received.  This is often handy during development to limit the amount of data consumed from a device that produces output continuously.  However, this approach is no good for devices like our charger that will produce a finite, indeterminate number of records - if the device stops sending records before the `count` has been reached, the loop will still block.  Instead, we could use :func:`pipecat.limit.duration` to limit the total amount of time the loop is allowed to run instead:

This approach is an improvement because it puts an upper-bound on the amount of time the loop will run, whether the device has stopped sending records or not.  However, it's still error-prone, since we don't know in advance how long charging will take - if we set the duration too low, it may stop the loop before charging is complete.  If we set the duration too high, we will capture all the records we want, but we will likely waste time waiting for records that will never come.  Ideally, we would like to exit the loop as soon as the charger tells us it's finished.  Fortunately, the charger provides a field - `charger/mode` that can do just that:

:func:`pipecat.limit.until` terminates the loop as soon as it receives a record with the given key and value.  This approach finally gets us our desired behavior (loop ends as soon as the charger is finished), but it could use just a little more work to make it robust.  For example, what happens if the charger stops sending data before the mode changes?  We could combine :func:`pipecat.limit.until` with :func:`pipecat.limit.duration`, but that would still suffer from the terminate-too-soon / waste-too-much-time problem.  Fortunately, we know from testing that our charger sends records every two seconds (if at all), and Pipecat provides :func:`pipecat.limit.timeout`, which can terminate the loop if it doesn't receive a record within a specified time interval: 

While this example exits normally, you can see how easy it is to combine limit functions to control when the `for` loop terminates.